# Benchmark model generation

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import os


# Views 3
import views_runs
from viewser.operations import fetch
from views_forecasts.extensions import *
from viewser import Queryset, Column


In [ ]:
# Common parameters:

dev_id = 'Fatalities002'
run_id = 'Fatalities002'
EndOfHistory = 508
get_future = False

username = os.getlogin()

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

fi_steps = [1,3,6,12,36]
# Specifying partitions

calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{username}/Dropbox (ViEWS)/ViEWS/'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/ViEWS predicting fatalities/Tables/'



print('Dropbox path set to',Mydropbox)
print('Overleaf path set to',overleafpath)

## cm level

### Based on constituent models

Short version, 22 models: 
1 "draw"
from each of 22 constituent models

Long version, 440 models:
20 "draws" from each of 22 constituent models, using predictions for adjacent steps (from s-4 to s+6). Some duplications to weight the most proximate steps more.



In [ ]:

# Fatalities002 stuff - contains the list of the current fatalities002 ensemble models

from ModelDefinitions import DefineEnsembleModels

level = 'cm'
ModelList_cm = DefineEnsembleModels(level)
    
i = 0
for model in ModelList_cm:
    print(i, model['modelname'], model['data_train'])
    i = i + 1

# Retrieving the predictions for calibration and test partitions
# The ModelList contains the predictions organized by model
from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated

ModelList_cm = RetrieveStoredPredictions(ModelList_cm, steps, EndOfHistory, dev_id, level, get_future)

ModelList_cm = CalibratePredictions(ModelList_cm, EndOfHistory, steps)

In [ ]:
ModelList_cm[0]['predictions_test_df']
ModelList_cm[0]

In [ ]:
# Dataframe with actuals
df_actuals = pd.DataFrame(ModelList_cm[0]['predictions_test_df']['ln_ged_sb_dep'])
print(df_actuals.head())
print(df_actuals.tail())


In [ ]:
def reshape_df_cm(df, draw):
    ''' Drops steps we will not need in the benchmark model. 
    Another round of drops are done below '''
    steps_to_drop = ['ln_ged_sb_dep','step_pred_23','step_pred_24',
                     'step_pred_25','step_pred_26','step_pred_27','step_pred_28','step_pred_29','step_pred_30',
                     'step_pred_31','step_pred_32','step_pred_33','step_pred_34','step_pred_35','step_pred_36',]
    df = df.drop(steps_to_drop,axis=1)
    df.reset_index(inplace=True)
    df['draw'] = draw
    df_long = pd.wide_to_long(df, 'step_pred_', i = ['month_id', 'country_id', 'draw'], j = 'step')
    return(df_long)
    
model_draw = 0
df = ModelList_cm[model_draw]['predictions_test_df'].copy()
df_cm_results_long = reshape_df_cm(df,model_draw)
print('Starting with model/draw',model_draw, model['modelname'])
print(df_cm_results_long.describe())


for model in ModelList_cm[1:]:
    model_draw += 1
    print('Appending model/draw',model_draw, model['modelname'])
    df = ModelList_cm[model_draw]['predictions_test_df'].copy()
    df_reshaped = reshape_df_cm(df,model_draw)
    df_cm_results_long = pd.concat([df_cm_results_long ,df_reshaped], axis=0)
    

In [ ]:
df_cm_results_long['prediction'] = np.round_(np.expm1(df_cm_results_long['step_pred_'])).astype('int32')
df_cm_results_long.drop(columns=['step_pred_'], inplace=True)
# Results file in long format
print(df_cm_results_long.describe())
print(df_cm_results_long.tail())

print(df_cm_results_long.loc[492].describe())

# Extending by copying adjacent steps

df_cm_results_extended=df_cm_results_long.copy()
print(df_cm_results_extended.describe())
print(df_cm_results_extended.head())

In [ ]:
df_cm_final_extended = df_cm_results_extended.copy()

def make_dfcopy(df_in, step, shifted_step, repetition):
    ''' Makes a 'copy' of the df with a shifted step '''
#    print(step, shifted_step, repetition)
    df = pd.DataFrame(df_in[df_in.index.get_level_values('step').isin([shifted_step])]).copy()
    df.reset_index(inplace = True)
    df['step'].replace(shifted_step, step, inplace = True)
#    print(df.describe())
    df['draw'] = (df['draw'] + len(ModelList) * repetition)
#    print(df.describe())
    df.set_index(['month_id', 'country_id', 'step', 'draw'], inplace=True)
    return(df)
    
df_list_steps = []
for step in range(3,14+1):     
#    print(80*'*')
    print('Step', step, '-- Original dataframe for step', step, 'is:')
    df = pd.DataFrame(df_cm_results_extended[df_cm_results_extended.index.get_level_values('step').isin([step])])
#    print(df.head(3))
    repetition = 1
    df_list = []
    for shift in [(-4,1),(-3,1),(-2,3),(-1,4),(0,3),(1,3),(2,2),(3,2),(4,1),(5,1),(6,1),(7,1),(8,1)]:
        for copy in range(1,shift[1]+1):
            shifted_step = step+shift[0]
            if shifted_step < 1:
                shifted_step = step
            step_list = [shifted_step]
            df = make_dfcopy(df_in = df_cm_results_extended,step = step, shifted_step = shifted_step, repetition = repetition)
            df_list.append(df)
            repetition += 1
    df_cm_temp = pd.concat(df_list)
    df_list_steps.append(df_cm_temp)

df_cm_final_extended = pd.concat(df_list_steps)
#df.reorder_levels(['month_id','country_id','steps','draw'])

In [ ]:
help(make_dfcopy)

In [ ]:
df_cm_final_extended.loc[492].describe()

In [ ]:
df_cm_final_extended.tail()

In [ ]:
# Pruning the original down to steps 3-14
steps_to_keep = range(3,15)

df_cm_results_long_pruned = df_cm_results_long[df_cm_results_long.index.get_level_values('step').isin(steps_to_keep)]
df_cm_results_long_pruned.tail()

### cm last historical values benchmark model

In [ ]:
qs = (Queryset("benchmark_cm", "country_month")

   # target variable
   .with_column(Column("ged_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                .transform.missing.fill()
                .transform.missing.replace_na()
                )


   .with_theme("benchmark")
   .describe("""Data for empirical benchmark model, cm level


            """)
   )

#queryset = Queryset("name", "loa") # if not already defined
column = "ged_sb_best_sum_nokgi"
table = "ged2_cm"
lags=range(1,65)
for lag in lags: 
    qs = qs.with_column(Column(column+'_' + str(lag), from_table = table, from_column=column)
                        .transform.missing.replace_na()
                        .transform.temporal.tlag(lag)
                       )
df_cm_historical_values = qs.publish().fetch()
df_cm_historical_values = df_cm_historical_values.loc[445:492]

df_cm_historical_values.describe()

In [ ]:
# Creating predictions for test partition
number_of_lags = 45
maxstep = 14
df_list_bystep = []
for step in range(3,maxstep+1):
    lags=range(step,step + number_of_lags)
    draw = 1
    print('step:',step,'lag:',lag, 'draw:',draw)
    df_list = []
    for lag in lags:
        number_of_repetitions = number_of_lags+step-lag
#        print('lag:',lag,'repetitions:',number_of_repetitions)
#        print('step:',step,'lag:',lag, 'draw:',draw)
        for repetition in range(1,number_of_repetitions):
            lagged_col = 'ged_sb_best_sum_nokgi_' + str(lag)
            df = pd.DataFrame(df_cm_historical_values[lagged_col].copy())
            df.reset_index(inplace=True)
            df['prediction'] = df[lagged_col]
#            print(df.head())
            df.drop(columns=[lagged_col], inplace=True)
            df['step'] = step
            df['draw'] = draw
            df.set_index(['month_id', 'country_id','draw', 'step'], inplace=True)
            df_list.append(df)
#            if draw == 1 and step == 1:
#                df_cm_predictions_historical_values = df.copy()
#            else:
#                df_cm_predictions_historical_values = pd.concat([df_cm_predictions_historical_values,df])
            draw = draw + 1
    df_cm_predictions_lag = pd.concat(df_list)
    df_list_bystep.append(df_cm_predictions_lag)
    print('Number of draws:',draw + 1)
df_cm_predictions_historical_values = pd.concat(df_list_bystep) 

df_cm_predictions_historical_values.describe()

In [ ]:
def aggregate_and_categorize(df, level):
    ''' This function aggregates the input df across all draws, and returns summary statistics for the prediction model '''
    if level == 'cm':
        index = ['month_id','country_id','step']
    if level == 'pgm':
        index = ['month_id', 'priogrid_gid','step']
    df_to_aggregate = df.copy()
    df_to_aggregate['log_prediction'] = np.log1p(df_to_aggregate['prediction'] )

    # Proportion of draws in fatality categories
    #for cutoffs in [0,1,10,100,1000,10000]:
    bins = pd.IntervalIndex.from_tuples([(-1, 0), (1, 10), (11, 100), (101, 1000), (1001, 10000), (10001,100000000)])
    df_to_aggregate['categorical'] = pd.cut(df_to_aggregate['prediction'],bins)
    df_to_aggregate_dummies = pd.get_dummies(df_to_aggregate['categorical'],prefix='cat')
    df_to_aggregate = pd.concat([df_to_aggregate,df_to_aggregate_dummies],axis=1)

    # Mean and standard deviation of log predictions
    df_aggregated = pd.DataFrame(df_to_aggregate['log_prediction'].groupby(level=index).mean())
    df_aggregated.rename(columns={'log_prediction':'mean_log_prediction'},inplace=True)
    df_aggregated['std_log_prediction'] = df_to_aggregate['log_prediction'].groupby(level=index).std()
    for col in ('cat_(-1, 0]','cat_(1, 10]','cat_(11, 100]','cat_(101, 1000]','cat_(1001, 10000]','cat_(10001, 100000000]'):
        df_aggregated[col] = df_to_aggregate[col].groupby(level=index).mean()
    return(df_aggregated)
    

In [ ]:
# Aggregate across draws/samples to extract means, standard deviations, and category probabilities
df_cm_predictions_historical_values_aggregated = aggregate_and_categorize(df_cm_predictions_historical_values,'cm')
df_cm_predictions_ensemble_aggregated = aggregate_and_categorize(df_cm_final_extended,'cm')

print(df_cm_predictions_historical_values_aggregated.describe())
print(df_cm_predictions_historical_values_aggregated.mean())

In [ ]:
# Draw from the aggregated 

In [ ]:
# Export to csv
filename = Mydropbox + 'Prediction_competition_2023/' + 'cm_actuals.csv'
df_actuals.to_csv(filename)
filename = Mydropbox + 'Prediction_competition_2023/' + 'cm_benchmark_ensemble_22.parquet'
df_cm_results_long_pruned.to_parquet(filename)
filename = Mydropbox + 'Prediction_competition_2023/' + 'cm_benchmark_ensemble_550.parquet'
df_cm_final_extended.to_parquet(filename)
filename = Mydropbox + 'Prediction_competition_2023/' + 'cm_benchmark_ensemble_550_aggregated.parquet'
df_cm_predictions_ensemble_aggregated.to_parquet(filename)
filename = Mydropbox + 'Prediction_competition_2023/' + 'cm_benchmark_historical_values.parquet'
df_cm_predictions_historical_values.to_parquet(filename)
filename = Mydropbox + 'Prediction_competition_2023/' + 'cm_predictions_historical_values_aggregated.parquet'
df_cm_predictions_historical_values_aggregated.to_parquet(filename)

# pgm benchmark

## Ensemble model pgm benchmark

kj

In [ ]:

level = 'pgm'
ModelList_pgm = DefineEnsembleModels(level)
    
i = 0
for model in ModelList_pgm:
    print(i, model['modelname'], model['data_train'])
    i = i + 1
    
# Retrieving the predictions for calibration and test partitions
# The ModelList contains the predictions organized by model
from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated

ModelList_pgm = RetrieveStoredPredictions(ModelList_pgm, steps, EndOfHistory, dev_id, level, get_future)

#ModelList_pgm = CalibratePredictions(ModelList_pgm, EndOfHistory, steps)

# Dataframe with actuals
df_actuals_pgm = pd.DataFrame(ModelList_pgm[0]['predictions_test_df']['ln_ged_sb_dep'])
print(df_actuals_pgm.head())
print(df_actuals_pgm.tail())


In [ ]:
# Reshaping
def reshape_df_pgm(df, draw):
    ''' Drops steps we will not need in the benchmark model. 
    Another round of drops are done below '''
    steps_to_drop = ['ln_ged_sb_dep','step_pred_23','step_pred_24',
                     'step_pred_25','step_pred_26','step_pred_27','step_pred_28','step_pred_29','step_pred_30',
                     'step_pred_31','step_pred_32','step_pred_33','step_pred_34','step_pred_35','step_pred_36',]
    df = df.drop(steps_to_drop,axis=1)
    df.reset_index(inplace=True)
    df['draw'] = draw
    df_long = pd.wide_to_long(df, 'step_pred_', i = ['month_id', 'priogrid_id', 'draw'], j = 'step')
    return(df_long)

model_draw = 0
df = ModelList_pgm[model_draw]['predictions_test_df'].copy()
df_pgm_results_long = reshape_df_pgm(df,model_draw)
print('Starting with model/draw',model_draw, model['modelname'])
print(df_pgm_results_long.describe())


for model in ModelList_pgm[1:]:
    model_draw += 1
    print('Appending model/draw',model_draw, model['modelname'])
    df = ModelList_pgm[model_draw]['predictions_test_df'].copy()
    df_reshaped = reshape_df_pgm(df,model_draw)
    df_pgm_results_long = pd.concat([df_pgm_results_long ,df_reshaped], axis=0)
    

In [ ]:
df_pgm_results_long['prediction'] = np.round_(np.expm1(df_pgm_results_long['step_pred_'])).astype('int32')
df_pgm_results_long.drop(columns=['step_pred_'], inplace=True)
# Results file in long format
print(df_pgm_results_long.describe())
print(df_pgm_results_long.tail())

print(df_pgm_results_long.loc[492].describe())

# Extending by copying adjacent steps

df_pgm_results_extended=df_pgm_results_long.copy()

In [ ]:
print(df_pgm_results_extended.describe())
print(df_pgm_results_extended.head())

In [ ]:
df_pgm_final_extended = df_pgm_results_extended.copy()

def make_dfcopy_pgm(df_in, step, shifted_step, repetition):
    ''' Makes a 'copy' of the df with a shifted step '''
#    print(step, shifted_step, repetition)
    df = pd.DataFrame(df_in[df_in.index.get_level_values('step').isin([shifted_step])]).copy()
    df.reset_index(inplace = True)
    df['step'].replace(shifted_step, step, inplace = True)
#    print(df.describe())
    df['draw'] = (df['draw'] + len(ModelList) * repetition)
#    print(df.describe())
    df.set_index(['month_id', 'priogrid_id', 'step', 'draw'], inplace=True)
    return(df)
    
df_list_steps_pgm = []
for step in range(3,14+1):     
#    print(80*'*')
    print('Step', step, '-- Original dataframe for step', step, 'is:')
    df = pd.DataFrame(df_pgm_results_extended[df_pgm_results_extended.index.get_level_values('step').isin([step])])
    print(df.head(3))
    repetition = 1
    df_list_pgm = []
    for shift in [(-4,1),(-3,1),(-2,3),(-1,4),(0,3),(1,3),(2,2),(3,2),(4,1),(5,1),(6,1),(7,1),(8,1)]:
        for copy in range(1,shift[1]+1):
            shifted_step = step+shift[0]
            if shifted_step < 1:
                shifted_step = step
            step_list = [shifted_step]
            df = make_dfcopy_pgm(df_in = df_pgm_results_extended,step = step, shifted_step = shifted_step, repetition = repetition)
            df_list_pgm.append(df)
            repetition += 1
    df_pgm_temp = pd.concat(df_list)
    df_list_steps_pgm.append(df_cm_temp)

#df_pgm_final_extended = pd.concat(df_list_steps)
#df.reorder_levels(['month_id','country_id','steps','draw'])

## Historical values pgm benchmark

In [ ]:
# lag settings (number of temporal lags at each spatial lag level)
tlags_cell = 40
tlags_firstorder = 27
tlags_secondorder = 21

# Spatial lags, first-order lag 1:
kernel_inner=1
kernel_width=1
kernel_power=0
norm_kernel=0

print('Retrieving data for inner cells')

column = "ged_sb_best_sum_nokgi"
table = "ged2_pgm"

qs = (Queryset("benchmark_pgm", "priogrid_month")

   # target variable at t0
   .with_column(Column("ged_sb", from_table="ged2_pgm", from_column="ged_sb_best_sum_nokgi")
                .transform.missing.fill()
                .transform.missing.replace_na()
                )
    # spatial lag at t0
   .with_column(Column("splag_ged_sb_0", from_table = table, from_column = column)
                     .transform.missing.replace_na()
                     .transform.spatial.lag(kernel_inner,kernel_width,kernel_power,norm_kernel)
                    )


   .with_theme("benchmark")
   .describe("""Data for empirical benchmark model, pgm level


            """)
   )

#queryset = Queryset("name", "loa") # if not already defined
column = "ged_sb_best_sum_nokgi"
table = "ged2_pgm"
tlags_0=range(1,tlags_cell + 1)
qs0 = qs.copy()
for lag in tlags_0: 
    qs0 = qs0.with_column(Column(column + '_' + str(lag), from_table = table, from_column=column)
                        .transform.missing.replace_na()
                        .transform.temporal.tlag(lag)
                       )

df_pgm_historical_values_0 = qs0.publish().fetch().loc[445:492]

# Spatial lags, first-order:
print('Retrieving data for first-order neighbors')


kernel_inner=1
kernel_width=1
kernel_power=0
norm_kernel=0


tlags_1=range(1,tlags_firstorder + 1)
qs1 = qs.copy()
for lag in tlags_1:
    qs1 = qs1.with_column(Column(column + '_splag_1_' + str(lag), from_table = table, from_column=column)
                        .transform.missing.replace_na()
                        .transform.temporal.tlag(lag)
                        .transform.spatial.lag(kernel_inner,kernel_width,kernel_power,norm_kernel)
                       )

df_pgm_historical_values_1 = qs1.publish().fetch().loc[445:492]

# Spatial lags; second-order:
print('Retrieving data for second-order neighbors')


kernel_inner=2
kernel_width=1
kernel_power=0
norm_kernel=0

tlags_2=range(1,tlags_secondorder + 1)
qs2 = qs.copy()
for lag in tlags_2:
    qs2 = qs2.with_column(Column(column + '_splag_2_' + str(lag), from_table = table, from_column=column)
                        .transform.missing.replace_na()
                        .transform.temporal.tlag(lag)
                        .transform.spatial.lag(kernel_inner,kernel_width,kernel_power,norm_kernel)
                       )

df_pgm_historical_values_2 = qs2.publish().fetch().loc[445:492]
print('Done retrieving data')

In [ ]:
df_pgm_historical_values_1.describe()

In [ ]:
# Merging the data frames
df_pgm_historical_values = pd.concat([df_pgm_historical_values_0, df_pgm_historical_values_1, df_pgm_historical_values_2], axis=1, ignore_index=False)

In [ ]:
df_pgm_historical_values = df_pgm_historical_values.loc[445:492]
# Computing averages from sums
for lag in tlags_1:
    col = column + '_splag_1_' + str(lag)
    df_pgm_historical_values[col] = df_pgm_historical_values[col]
for lag in tlags_2:
    col = column + '_splag_2_' + str(lag)
    df_pgm_historical_values[col] = df_pgm_historical_values[col]



df_pgm_historical_values.describe()

In [ ]:
df_pgm_historical_values_2.describe()

In [ ]:
# Creating predictions for test partition
number_of_lags_inner = 24
number_of_lags_1 = 12
number_of_lags_2 = 6
maxstep = 14
df_list_bystep = []
for step in range(3,maxstep+1):
    lags=range(step,number_of_lags_inner+step)
    draw = 1
#    print('step:',step,'lag:',lag, 'draw:',draw)
    df_list = []
    for lag in lags:
        for coltype in [('ged_sb_best_sum_nokgi_',number_of_lags_inner),('ged_sb_best_sum_nokgi_splag_1_',number_of_lags_1),('ged_sb_best_sum_nokgi_splag_2_',number_of_lags_2)]:
            number_of_repetitions = coltype[1]+step-lag
            for repetition in range(1,number_of_repetitions+1):
                if lag <= coltype[1] + step:
                    lagged_col = coltype[0] + str(lag)
#                    print('draw:',draw, 'step:', step, 'lag:',lag,'repetition:', repetition, 'colname:', lagged_col)
                    df = pd.DataFrame(df_pgm_historical_values[lagged_col].copy())
                    df.reset_index(inplace=True)
                    df['prediction'] = df[lagged_col].astype('int32')
                    df.drop(columns=[lagged_col], inplace=True)
                    df['step'] = step
                    df['draw'] = draw
                    df.set_index(['month_id', 'priogrid_gid', 'step','draw'], inplace=True)
                    df_list.append(df)
                    draw = draw + 1
    print('Concatenating', draw-1, 'repetitions for step', step)
    df_pgm_predictions_lag = pd.concat(df_list)
    df_list_bystep.append(df_pgm_predictions_lag)
    
#df_pgm_predictions_historical_values = pd.concat(df_list_bystep) 
#df_pgm_predictions_historical_values.describe()

In [ ]:
df_list_bystep[0].dtypes

In [ ]:
# Concatenating step-level dataframes
single_file = False

if single_file:
    df_pgm_predictions_historical_values = df_list_bystep[0]
    list_item = 1
    for step in range(3+1,maxstep+1):
        print('adding data for step', step)
        df_pgm_predictions_historical_values = pd.concat([df_pgm_predictions_historical_values,df_list_bystep[list_item]])
        list_item += 1

    df_pgm_predictions_historical_values.describe()

In [ ]:
# Dataframe with actuals
df_actuals_pgm = pd.DataFrame(df_pgm_historical_values_0['ged_sb'])
print(df_actuals_pgm.head())
print(df_actuals_pgm.tail())

In [ ]:
# Export to csv
filename = Mydropbox + 'Prediction_competition_2023/' + 'pgm_actuals.parquet'
df_actuals_pgm.to_parquet(filename)

In [ ]:
# Aggregate across draws/samples to extract means, standard deviations, and category probabilities
# export to parquet step by step
step = 3
for df in df_list_bystep:
    print(step)
    filename = Mydropbox + 'Prediction_competition_2023/' + 'pgm_benchmark_historical_values_step_' + str(step) + '.parquet'
    df.to_parquet(filename)
    filename = Mydropbox + 'Prediction_competition_2023/' + 'pgm_benchmark_historical_values_step_' + str(step) + '_aggregated.parquet'
    df_aggregated = aggregate_and_categorize(df,'pgm')
    df_aggregated.to_parquet(filename)

    print(df_aggregated.describe())
    print(df_aggregated.mean())
        
    step = step+1

In [ ]:
filename = Mydropbox + 'Prediction_competition_2023/' + 'pgm_benchmark_historical_values.parquet'
df_from_file = pd.read_parquet(filename)

In [ ]:
filename = Mydropbox + 'Prediction_competition_2023/' + 'pgm_benchmark_historical_values_step_3.parquet'
df_from_file = pd.read_parquet(filename)

In [ ]:
df_from_file